In [1]:
# Import parquet file with pySpark
from pyspark.sql import SparkSession

# Create a spark session
spark = SparkSession.builder.appName("parquet").getOrCreate()

# Read parquet file
df = spark.read.parquet(r"./scaled_simplified_train_data.parquet")

# Show the data
df.show()


+------------+-------------------+------------------+--------------------+-------------------------+--------------------+------------------------+------------------------+-----------------------+--------------------------+------------------------+---------------------------+--------------------+
|attack_index|     dst_subnet_vec| src_subnet_hashed| scaled_ttl_features|scaled_tcp_flags_features|scaled_flow_features|scaled_duration_features|scaled_pkt_size_features|scaled_tcp_win_features|scaled_throughput_features|scaled_protocol_features|scaled_l4_dst_port_features| scaled_dns_features|
+------------+-------------------+------------------+--------------------+-------------------------+--------------------+------------------------+------------------------+-----------------------+--------------------------+------------------------+---------------------------+--------------------+
|         0.0|(2831,[1018],[1.0])|(1024,[597],[0.0])|[-0.6995209701154...|     [-0.8693157631485...|[-0.36765

In [ ]:
from typing import Protocol
from pyspark.ml.feature import VectorAssembler, StandardScaler, FeatureHasher, OneHotEncoder
from pyspark.ml import Pipeline

# OneHot encoding for destination subnet since higher cardinality
dst_subnet_encoder = OneHotEncoder(inputCol="dst_subnet_index", outputCol="dst_subnet_vec")

# Source address
# Feature hashing for source subnet since lower cardinality
src_subnet_hasher = FeatureHasher(inputCols=["src_subnet_index"], outputCol="src_subnet_hashed", numFeatures=1024)

# Assemble numerical features for bytes and packets trasmitted
trasimitted_in_features = ["IN_BYTES", "IN_PKTS"]
trasmitted_out_features = ["OUT_BYTES", "OUT_PKTS"]
retransimtted_in_features = ["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"]
retransimtted_out_features = ["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"]
throughput_in_features = ["SRC_TO_DST_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT"]
throughput_out_features = ["DST_TO_SRC_AVG_THROUGHPUT", "DST_TO_SRC_SECOND_BYTES"]
ttl_features = ["MIN_TTL", "MAX_TTL"]
tcp_flags_features = ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"]
icmp_features = ["ICMP_TYPE", "ICMP_IPV4_TYPE"]
flow_max_features = ["LONGEST_FLOW_PKT", "MAX_IP_PKT_LEN"]
flow_min_features = ["SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN"]
flow_duration_features = ["FLOW_DURATION_MILLISECONDS"]
tcp_in_features = ["TCP_WIN_MAX_IN"]
tcp_out_features = ["TCP_WIN_MAX_OUT"]
duration_in_features = ["DURATION_IN"]
duration_out_features = ["DURATION_OUT"]
protocol_features = ["PROTOCOL"]
l4_src_port_features = ["L4_SRC_PORT"]
l4_dst_port_features = ["L4_DST_PORT"]
dns_features = ["DNS_QUERY_ID", "DNS_QUERY_TYPE", "DNS_TTL_ANSWER"]
ftp_features = ["FTP_COMMAND_RET_CODE"]
l7_proto_features = ["L7_PROTO"]
num_packets_by_size_features = ["NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES"]



# Assemble numerical features for flow duration
flow_duration_assembler = VectorAssembler(
    inputCols=["FLOW_DURATION_MILLISECONDS"],
    outputCol="flow_duration_feature"
)

tcp__assembler = VectorAssembler(
    inputCols=["TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"],
    outputCol="tcp_win_features",
)

# Apply StandardScaler to the assembled vectors
bytes_pkts_scaler = StandardScaler(
    inputCol="bytes_pkts_features", 
    outputCol="scaled_bytes_pkts_features", 
    withStd=True, 
    withMean=True
)

flow_duration_scaler = StandardScaler(
    inputCol="flow_duration_feature", 
    outputCol="scaled_flow_duration", 
    withStd=True, 
    withMean=True
)

throughput_scaler = StandardScaler(
    inputCol="throughput_features", 
    outputCol="scaled_throughput_features", 
    withStd=True, 
    withMean=True
)

pkt_len_scaler = StandardScaler(
    inputCol="pkt_len_features", 
    outputCol="scaled_pkt_len_features", 
    withStd=True, 
    withMean=True
)

tcp_win_scaler = StandardScaler(
    inputCol="tcp_win_features", 
    outputCol="scaled_tcp_win_features", 
    withStd=True, 
    withMean=True
)

# Create a pipeline for scaling
scaling_pipeline = Pipeline(stages=[
    dst_subnet_encoder,
    src_subnet_hasher,
    bytes_pkts_assembler, 
    bytes_pkts_scaler, 
    flow_duration_assembler, 
    flow_duration_scaler, 
    throughput_assembler, 
    throughput_scaler, 
    pkt_len_assembler, 
    pkt_len_scaler, 
    tcp_win_assembler, 
    tcp_win_scaler
])

# Fit the scalling pipeline to the encoded training data
scalling_model = scaling_pipeline.fit(encoded_train_data)

# Transform both encoded training and test data
processed_train_data = scalling_model.transform(encoded_train_data)
processed_test_data = scalling_model.transform(encoded_test_data)

In [3]:
from pyspark.ml.feature import VectorAssembler

# Preparar os dados com VectorAssembler
feature_columns = df.columns
feature_columns.remove("attack_index")
print(feature_columns)
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_assembled = assembler.transform(df)

['dst_subnet_vec', 'src_subnet_hashed', 'scaled_ttl_features', 'scaled_tcp_flags_features', 'scaled_flow_features', 'scaled_duration_features', 'scaled_pkt_size_features', 'scaled_tcp_win_features', 'scaled_throughput_features', 'scaled_protocol_features', 'scaled_l4_dst_port_features', 'scaled_dns_features']


In [4]:
from pyspark.ml.classification import RandomForestClassifier, NaiveBayes, LogisticRegression, MultilayerPerceptronClassifier

# Definir o Random Forest
rf = RandomForestClassifier(labelCol="attack_index", featuresCol="features", numTrees=5)

# Definir o Naive Bayes
#nb = NaiveBayes(labelCol="attack_index", featuresCol="features")

# Definir o Logistic Regression
lr = LogisticRegression(labelCol="attack_index", featuresCol="features")

# Configuração do Multilayer Perceptron
# Obter o número de características
def get_feature_count(df, feature_col="features"):
# Extrai os metadados da coluna de características e calcula a soma dos tamanhos dos atributos
    attributes = df.schema[feature_col].metadata["ml_attr"]["attrs"]
    feature_count = sum(len(attrs) for attrs in attributes.values())
    return feature_count
input_layers = get_feature_count(df_assembled, "features")
output_layers = df_assembled.select("attack_index").distinct().count()
hidden_layers = [input_layers // 2, output_layers]
mlp = MultilayerPerceptronClassifier(
    labelCol="attack_index", 
    featuresCol="features", 
    layers=hidden_layers, 
    maxIter=50
)

In [5]:
# Random Forest
model_rf = rf.fit(df_assembled)
#model_nb = nb.fit(df_assembled)
# logistic regression
model_lr = lr.fit(df_assembled)
# Multilayer Perceptron
model_mlp = mlp.fit(df_assembled)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/IPyt

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Avaliador
evaluator = MulticlassClassificationEvaluator(labelCol="attack_index", predictionCol="prediction", metricName="accuracy")

# Avaliar Random Forest
accuracy_rf = evaluator.evaluate(model_rf.transform(df_assembled))
print(f"Random Forest Accuracy: {accuracy_rf}")

# Avaliar Naive Bayes
#accuracy_nb = evaluator.evaluate(model_nb.transform(df_assembled))
#print(f"Naive Bayes Accuracy: {accuracy_nb}")

# Avaliar Logistic Regression
accuracy_lr = evaluator.evaluate(model_lr.transform(df_assembled))
print(f"Logistic Regression Accuracy: {accuracy_lr}")

# Avaliar Multilayer Perceptron
accuracy_mlp = evaluator.evaluate(model_mlp.transform(df_assembled))
print(f"Multilayer Perceptron Accuracy: {accuracy_mlp}")